In [38]:
from dotenv import load_dotenv

load_dotenv()

True

In [39]:
!pip install pymupdf

### 1. 문서 로드 (load Documents)

In [40]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/snow-white.pdf")

docs = loader.load()

In [41]:
print(docs[5].page_content)

왕자는깨어난백설공주를보고기뻐했어요.
“공주님, 나는이웃나라왕자입니다.”
“왕자님이나를다시살려주셨군요.”
“나와결혼해주시겠어요?”
“네, 좋아요!”
두사람은일곱난쟁이와함께오래오래행복하게살
았답니다.



In [42]:
print(docs[0].__dict__)

{'id': None, 'metadata': {'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, 'page_content': '백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.\n하지만기쁨도잠시, 왕비는곧세상을떠나고말았어\n요.\n', 'type': 'Document'}


## 2. 문서 분할(Split Documents)

In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=10)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의수 : {len(split_documents)}")


분할된 청크의수 : 21


### 3. 임베딩(Embeding 생성)

In [44]:
from langchain_openai import OpenAIEmbeddings

embedings = OpenAIEmbeddings()

4. DB 생성(벡터스토어 생성) 및 저장
* FAISS(Facebook AI Similarity Search)
    * 페이스북에서 개발한 유사도 검색 및 클러스터링 라이브러리
    * 벡터 데이터셋에서 빠른 유사도 검색

In [45]:
!pip install faiss-cpu

In [58]:
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=split_documents, embeddings=embedings)


TypeError: VectorStore.from_documents() missing 1 required positional argument: 'embedding'

### 5.검색기(Retriever) 생성

In [60]:
# 벡터스토어에 있는 정보를 검색하고 생성
retriver = vectorstore.as_retriever()

NameError: name 'vectorstore' is not defined

In [61]:
retriver.invoke("백설공주와 일곱난쟁이는 어디서 만났어?")

NameError: name 'retriver' is not defined

### 6.프롬프트 생성

당신은 질문-답변 작업을 위한 어시스턴트입니다.
주어진 문맥을 사용하여 질문에 답변하세요.
유치원 선생님이 아이에게 말하는 것처럼 매우 칠절하고 부드러운 어조
따뜻하고 친근한 방식으로
답을 모르는 경우에는 모른다고 답변
한국어로 답변

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    당신은 질문-답변 작업을 위한 어시스턴트입니다.
주어진 문맥을 사용하여 질문에 답변하세요.
유치원 선생님이 아이에게 말하는 것처럼 매우 칠절하고 부드러운 어조
따뜻하고 친근한 방식으로
답을 모르는 경우에는 모른다고 답변
한국어로 답변
    """
)